In [13]:
# 1. 導入必要的套件
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain_community.utilities import SerpAPIWrapper
from langchain.tools import Tool
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
# 2. 設置環境
import os
from dotenv import load_dotenv
load_dotenv()

# 3. 初始化 LLM
llm = ChatOpenAI(model_name="gpt-4-turbo-preview", temperature=0.1)

# 4. 設置搜尋工具
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="用於搜尋即時資訊的工具"
    ),
]

# 5. 設置記憶存儲
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="output"  # 添加輸出鍵
)

# 6. 創建整合的 prompt 模板
prefix = """你是一個投資顧問助手。使用以下工具來回答人類的問題。請使用繁體中文回答。

請注意：
1. 回答問題時要參考對話歷史中的相關資訊
2. 如果問題提到"根據前面的資訊"，要特別注意使用之前對話中提到的數據
3. 給出投資建議時，要基於具體的數據和分析"""

format_instructions = """回答時請遵循以下格式：

Question: 輸入的問題
Thought: 你應該思考要做什麼
Action: {tool_names} 中的一個工具名稱
Action Input: 工具的輸入
Observation: 工具的結果
... (這個 Thought/Action/Action Input/Observation 可以重複)
Thought: 我現在知道最終答案
Final Answer: 用繁體中文回答原始問題"""

suffix = """先前對話歷史：
{chat_history}

Question: {input}
{agent_scratchpad}"""

# 7. 創建 prompt 和 agent
prompt = ZeroShotAgent.create_prompt(
    tools=tools,
    prefix=prefix,
    suffix=suffix,
    format_instructions=format_instructions,
    input_variables=["input", "chat_history", "agent_scratchpad"]
)

llm_chain = LLMChain(llm=llm, prompt=prompt)
agent = ZeroShotAgent(
    llm_chain=llm_chain,
    tools=tools,
    verbose=True
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory,
    max_iterations=3,  # 限制最大迭代次數
    early_stopping_method="generate"  # 使用生成式早停
)

# 8. 測試對話
def chat_with_agent(question):
    try:
        response = agent_executor.invoke({
            "input": question
        })
        return response.get("output", "無法獲得回答")
    except Exception as e:
        return f"發生錯誤：{str(e)}"

In [14]:
# 9. 執行測試
question1 = "鈊象電子最近的營收狀況如何？"
print("問題1:", question1)
print("AI回答:", chat_with_agent(question1))

question2 = "根據前面的資訊，你覺得這家公司值得投資嗎？"
print("\n問題2:", question2)
print("AI回答:", chat_with_agent(question2))

問題1: 鈊象電子最近的營收狀況如何？


> Entering new AgentExecutor chain...
Thought: 為了回答這個問題，我需要查找鈊象電子最近的營收報告或相關新聞。

Action: Search
Action Input: 鈊象電子 最近 營收
Observation: {'type': 'organic_result', 'title': '月營收 - 鈊象電子', 'source': '鈊象電子股份有限公司', 'description': '2024'}
Thought:我需要更具體的信息來回答這個問題，因為目前的搜索結果沒有提供足夠的細節。

Action: Search
Action Input: 鈊象電子 2023 營收報告
Observation: ['鈊象电子近十二个月的营收17.241十亿; December 2019至2023的财年鈊象电子的营收为10.225十亿; December 2019至2023的财年，鈊象电子的营收中值为11.286十亿。', '遊戲股王鈊象（3293）6日公告去年12月營收17.1億元，月增2.3%、年增29.1%，單月營收再創歷史新高；累計去年度營收185.12億元，年增30.6%，創歷史新高。', '月營收 · 選擇「營運概況」選項 · 點選「每月營收」選擇「採用IFRSs後之月營業收入資訊」 · 下拉選擇「歷史資料」選項 · 請輸入公司代號：3293 · 輸入欲查詢年度、月份 · 按「查詢 ...', '遊戲股王鈊象電子（3293）公告10月自結損益。10月營收為16.8億元，年增3成（30.25％），單月稅前每股盈餘3.6元。值得一提的是，海外市場的營收授權的占比已從 ...', '鈊象(3293)每月營收 ; 2023/12, 1,324,622, 7.29 ; 2023/11, 1,234,535, 0.26 ; 2023/10, 1,231,274, 1.99 ; 2023/09, 1,207,190, -1.7 ...', '游戏日报讯3月11日消息 鈊象电子（IGS）日前披露的2023年财务信息显示，公司2023年营业收入为141.76亿元新台币（约合人民币32.38亿元），净利润为64.26亿元 ...', '財務報表 月營收. 股東專欄. 股東常會 歷年股利 